In [39]:
import pandas as pd

# 1. Data

In [40]:
path = "datas/tmdb_5000/tmdb_5000_movies.csv"

df = pd.read_csv(path)
crow, ccol = df.shape
print("[count] rows : {}, columns : {}".format(crow, ccol))
df.head(1)

[count] rows : 4803, columns : 20


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


# 2. Data Preprocessing

## 1. Column for using

In [41]:
cols = ['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'keywords', 'overview']
data = df[cols]

crow, ccol = data.shape
print("[count] rows : {}, columns : {}".format(crow, ccol))
data.head(1)

[count] rows : 4803, columns : 8


,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


## 2. Vote Preprocessing
- 평균평점이 불공정하게 처리되어 있음.
- weight rating으로 평점을 다시 처리해주는 작업

In [42]:
# 500위 안에 데이터가 들어와야 할 때의 vote_count
m = data['vote_count'].quantile(0.9)
print("vote_count : {}".format(m))

data = data.loc[data['vote_count'] >= m]
crow, ccol = data.shape
print("[count] rows : {}, columns : {}".format(crow, ccol))
data.head(1)

vote_count : 1838.4000000000015
[count] rows : 481, columns : 8


,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [43]:
c = data['vote_average'].mean()
print("vote_average : {}".format(c))

vote_average : 6.9629937629937615


In [44]:
def weight_rating(x, m=m, c=c):
    v = x['vote_count']
    r = x['vote_average']
    
    return (v / (v + m) * r) + (m / (v + m) * c)

In [45]:
data['score'] = data.apply(weight_rating, axis=1)
crow, ccol = data.shape
print("[count] rows : {}, columns : {}".format(crow, ccol))
data.head(1)

[count] rows : 481, columns : 9


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053


## 3. Column Extraction

In [46]:
import ast
# ast.literal_eval : 문자열로 표현된 실제 식으로 표현해주는 기능
exam = ast.literal_eval("{'a': 1234, 'b': 'hello'}")
print(exam)

{'a': 1234, 'b': 'hello'}


In [47]:
ex_col = ['genres', 'keywords']

for c in ex_col:
    data[c] = data[c].apply(ast.literal_eval)
    
# 문자열로 정의된 dict를 실제 dict로 바꾸는 작업
data[ex_col].head(1)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."


In [48]:
name_parse = lambda x: [d['name'] for d in x]
str_join = lambda x: " ".join(x)

for c in ex_col:
    data[c] = data[c].apply(name_parse).apply(str_join)
    
data.head(5)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,Action Adventure Crime,6.3,4466,107.376788,Spectre,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.493333
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.492998
4,49529,Action Adventure Science Fiction,6.1,2124,43.926995,John Carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.500396


# 3. Contents Based Filtering

## 1. Genre Vectorization

### 1. sklearn.feature_extraction.text.CountVectorizer 간단 소개

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

# ngram_range = (1,3)
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]
vect = CountVectorizer()
# 1. 훈련 시키면 문서에 존재하는 각 각의 단어에 토큰을 부여한다.
vect.fit(corpus)
print(vect.vocabulary_)

# 2. 출현빈도 카운트, BoW(Bag of Words) Encoding Vector 반환
# This의 토큰(9)에 True(1) 처리, is의 토큰(3)에 True 처리, the의 토큰(7)에 True 처리, second의 토큰(6)에 True 처리
# document의 토큰(1)에 True(1) 처리
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# [0, 1, 0, 1, 0, 0, 1, 1, 0, 1]
print(vect.transform(['This is the second document.']).toarray())
print(vect.transform(['i am th']).toarray())

{'this': 9, 'is': 3, 'the': 7, 'first': 2, 'document': 1, 'second': 6, 'and': 0, 'third': 8, 'one': 5, 'last': 4}
[[0 1 0 1 0 0 1 1 0 2]]
[[0 0 0 0 0 0 0 0 0 0]]


### 2. 예제 적용

In [53]:
vect = CountVectorizer(ngram_range=(1,3))

print("Input Value : {} ...".format(data['genres'].values[:2]))
vect.fit(data['genres'])
print("Genre word by Token : {} ...\n".format(str(vect.vocabulary_)[:80]))

# 데이터 적용, BoW(Bag of Words) Encoding Vector 변환
genre_vector = vect.transform(data['genres']).toarray()
print("title | encoding vector")
for idx, title in enumerate(data['title'][:4]):
    print(title, genre_vector[idx][:20])

Input Value : ['Action Adventure Fantasy Science Fiction' 'Adventure Fantasy Action'] ...
Genre word by Token : {'action': 0, 'adventure': 48, 'fantasy': 225, 'science': 326, 'fiction': 259, ' ...

title | encoding vector
Avatar [1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Pirates of the Caribbean: At World's End [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Spectre [1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The Dark Knight Rises [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]


## 2. Similarity Value Extraction

In [54]:
from sklearn.metrics.pairwise import cosine_similarity as cos

# argsort 는 각 각의 영화들의 모든 영화들과의 배열안에서 적용되며, 오름차순으로 정렬된다.
# cosine_similarity는 높을수록 유사도가 높은 것이기 때문에 ::-1로 모든 행에 대하여 reverse를 적용해준다.
# 각 각의 영화는 자신과 장르상으로 유사한 영화들의 인덱스를 순위로 가지고 있게 되는 것
sim_genre = cos(genre_vector, genre_vector).argsort()[:,::-1]
crow, ccol = sim_genre.shape
print("[count] rows : {}, columns : {}".format(crow, ccol))
sim_genre

[count] rows : 481, columns : 481


array([[  0,  13,  42, ..., 298, 297, 240],
       [ 11,   1, 200, ..., 329, 330, 240],
       [  2, 376, 216, ..., 314, 304, 240],
       ...,
       [478, 187,  12, ..., 326, 327,   0],
       [479, 466, 383, ..., 220, 224,   0],
       [480, 468, 294, ..., 246, 248,   0]])

In [76]:
data.iloc[sim_genre[3]].head(4)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.492998
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
3234,242582,Crime Drama Thriller,7.6,3395,89.095538,Nightcrawler,journalism underground tv station sociopath ho...,"When Lou Bloom, desperate for work, muscles in...",7.376231


## 3. Usage

In [77]:
def recommend_movie_list(df, movie_title, top=30):
    idx = data[data['title'] == title].index[0]
    
    # sim_idx : 위에서 구한 장르 유사도 데이터 이용
    sim_idx = sim_genre[idx][1:top + 1]
    
    return data.iloc[sim_idx]

In [78]:
# score 내림차순 정렬 수행
ran_movie = data.sample().iloc[0]

print("영화 '{}'에 대한 추천 데이터".format(ran_movie['title']))
reco_data = recommend_movie_list(data, movie_title=ran_movie['title']).sort_values('score', ascending=False)
reco_data.head(3)

영화 'Ex Machina'에 대한 추천 데이터


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,12002,187.322927,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138


## 4. 장르 빈도 확인

In [79]:
import numpy as np

features = vect.get_feature_names_out()
count = vect.transform(reco_data['genres']).toarray().sum(axis=0)

count_df = pd.DataFrame(np.column_stack([features, count]),columns=['genres (1,3)', 'count'])

print("영화 '{}'의 장르 데이터".format(ran_movie['title']))
print(ran_movie['genres'])
count_df.sort_values(['count'], ascending=False)[:10]

영화 'Ex Machina'의 장르 데이터
Drama Science Fiction


,"genres (1,3)",count
336,thriller,24
159,drama,22
143,crime,21
0,action,19
193,drama thriller,16
15,action crime,12
149,crime drama,9
152,crime drama thriller,5
158,crime thriller,5
18,action crime thriller,5
